In [1]:
import torch
import torch.optim as optim 
from torch.utils.tensorboard import SummaryWriter

from active_rl.networks import DQN
from active_rl.trainer import train_standard
from active_rl.utils import make_raw_env, env_shape, ReplayMemory, standard_DDQN, ActionSelector

In [2]:
BATCH_SIZE = 128
LR = 0.0000625
GAMMA = 0.99
EPS_START = 1.
EPS_END = 0.1
EPS_DECAY = 1000000 
NUM_STEPS = 20000000
MEMORY_CAPACITY = 800000
INITIAL_STEPS=200
TRAIN_FREQ = 4
TARGET_UPDATE_FREQ = 4000
EVAL_FREQ = 8000
SAVE_FREQ = 50000

ENV_NAME = 'Breakout'
NAME = 'build_ddqn_model_try4'
FILENAME = 'models/DDQN_breakout_model'

In [3]:
env_raw = make_raw_env(ENV_NAME)
channel, height, width, n_actions = env_shape(env_raw)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
policy_net = DQN(n_actions).to(device)
target_net = DQN(n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.Adam(policy_net.parameters(), lr=LR, eps=1.5e-4)
train_func = standard_DDQN
memory = ReplayMemory(MEMORY_CAPACITY, [5 * channel, height, width], n_actions) 
action_selector = ActionSelector(policy_net, EPS_START, EPS_END, EPS_DECAY, n_actions, device=device)
writer = SummaryWriter(f'runs/{NAME}')

In [ ]:
train_standard(policy_net, target_net, optimizer, memory, action_selector, TRAIN_FREQ, TARGET_UPDATE_FREQ,
               train_func, env_raw, NUM_STEPS, INITIAL_STEPS, EVAL_FREQ, filename=FILENAME, save_freq=SAVE_FREQ,
               writer=writer)